### Read the case, department, and source data into their own spark dataframes.

In [1]:
# import spark session
from pyspark.sql import SparkSession

# import spark functions
import pyspark.sql.functions as f

# import spark structure
from pyspark.sql.types import StructType, StructField, StringType

In [2]:
# create spark session
spark = SparkSession.builder.getOrCreate()

In [3]:
# read in data frames
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)

In [4]:
source.show(10)

+---------+-------------------+
|source_id|    source_username|
+---------+-------------------+
|   100137|   Merlene Blodgett|
|   103582|        Carmen Cura|
|   106463|    Richard Sanchez|
|   119403|     Betty De Hoyos|
|   119555|     Socorro Quiara|
|   119868|Michelle San Miguel|
|   120752|     Eva T. Kleiber|
|   124405|          Lori Lara|
|   132408|      Leonard Silva|
|   135723|       Amy Cardenas|
+---------+-------------------+
only showing top 10 rows



In [5]:
dept.show(10)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

In [18]:
case.show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05 08:30:00  
 case_late            | NO                   
 num_days_late        | -2.0126041

### Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [7]:
source.write.json("source_to_json", mode="overwrite")

source.write.csv("source_to_csv", mode="overwrite")

Inspect your folder structure. 
What do you notice?
Inspect the data in your dataframes. 
Are the data types appropriate? 

### Write the code necessary to cast the values to the appropriate types.

In [8]:
source.show(1)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [9]:
source = (source.withColumn('source_id', f.col('source_id').cast('string'))
                .withColumn('source_name', f.col('source_id').cast('string')))

### How old is the latest (in terms of days past SLA) currently open issue?

In [10]:
case.select(f.round(f.max(case.num_days_late)).alias('max_days_late')).show()

+-------------+
|max_days_late|
+-------------+
|        520.0|
+-------------+



### How long has the oldest (in terms of days since opened) currently opened issue been open?

In [11]:
(case.filter(case.case_closed != "YES")
     .select(f.round(f.max(case.num_days_late)).alias('max_days_late'))
     .show())

+-------------+
|max_days_late|
+-------------+
|        349.0|
+-------------+



### How many Stray Animal cases are there?

In [12]:
case.show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [13]:
(case.filter(case.service_request_type == "Stray Animal")
 
 .select(f.round(f.count(case.num_days_late)).alias('stray_animal_cases'))

.show())

+------------------+
|stray_animal_cases|
+------------------+
|             26760|
+------------------+



### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [14]:
(case.filter(case.dept_division  == "Field Operations")
     .filter(case.service_request_type  != "Officer Standby")
     .select(f.round(f.count(case.dept_division)).alias('number_of_cases'))
     .show()
 )

+---------------+
|number_of_cases|
+---------------+
|         113902|
+---------------+



### Convert the council_district column to a string column.

In [15]:
case.withColumn('council_district', case.council_district.astype('string')).show(1, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



### Extract the year from the case_closed_date column.

In [16]:
fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", f.to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", f.to_timestamp("case_closed_date", fmt))
    .withColumn("SLA_due_date", f.to_timestamp("SLA_due_date", fmt))
)

In [17]:
case.withColumn('year_close', f.year("case_closed_date")).show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 year_close           | 2018                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05 08:30:00  
 case_late            | NO        

### Convert num_days_late from days to hours in new columns num_hours_late.

In [21]:
case.withColumn('num_hours_late', f.expr('round(num_days_late * 24)')).show(2, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 num_hours_late       | -23964.0             
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 SLA_due_date         | 2018-01-05 08:30:00  
 case_late            | NO        

### Join the case data with the source and department data.

In [22]:
source.show(1)

+---------+----------------+-----------+
|source_id| source_username|source_name|
+---------+----------------+-----------+
|   100137|Merlene Blodgett|     100137|
+---------+----------------+-----------+
only showing top 1 row



In [23]:
dept.show(1)

+---------------+----------------+----------------------+-------------------+
|  dept_division|       dept_name|standardized_dept_name|dept_subject_to_SLA|
+---------------+----------------+----------------------+-------------------+
|311 Call Center|Customer Service|      Customer Service|                YES|
+---------------+----------------+----------------------+-------------------+
only showing top 1 row



In [30]:
(source
        .join(case,"source_id","left")
        .join(dept,"dept_division","left").show(1, vertical = True))

-RECORD 0--------------------------------------
 dept_division          | Waste Collection     
 source_id              | 136202               
 source_username        | Michelle Urrutia     
 source_name            | 136202               
 case_id                | 1014128056           
 case_opened_date       | 2018-01-02 08:21:00  
 case_closed_date       | 2018-01-10 08:39:00  
 SLA_due_date           | 2018-01-05 08:30:00  
 case_late              | YES                  
 num_days_late          | 5.00681713           
 case_closed            | YES                  
 service_request_type   | Solid Waste Fees ... 
 SLA_days               | 3.00619213           
 case_status            | Closed               
 request_address        | 3214  STONEY FORK... 
 council_district       | 10                   
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
only showing top 1 row



### Are there any cases that do not have a request source?

In [31]:
df = (source.join(case,"source_id","left")
            .join(dept,"dept_division","left"))

In [33]:
df.filter('request_address is NULL').show(5,vertical = True)

(0 rows)



### What are the top 10 service request types in terms of number of requests?

In [44]:
(df.groupBy('service_request_type').agg(f.count('source_id').alias('number_of_requests'))
  .sort(f.col('number_of_requests').desc()).show(10))

+--------------------+------------------+
|service_request_type|number_of_requests|
+--------------------+------------------+
|           No Pickup|             89210|
|Overgrown Yard/Trash|             66403|
|        Bandit Signs|             32968|
|        Damaged Cart|             31163|
|Front Or Side Yar...|             28920|
|        Stray Animal|             27361|
|Aggressive Animal...|             25492|
|Cart Exchange Req...|             22608|
|Junk Vehicle On P...|             21649|
|     Pot Hole Repair|             20827|
+--------------------+------------------+
only showing top 10 rows



### What are the top 10 service request types in terms of average days late?

In [47]:
(df.groupBy('service_request_type').agg(f.mean('num_days_late').alias('avg_days_late'))
   .sort(f.col('avg_days_late').desc()).show(10))

+--------------------+------------------+
|service_request_type|     avg_days_late|
+--------------------+------------------+
|  Zoning: Junk Yards|175.95636210420932|
|Labeling for Used...|162.43032902285717|
|Record Keeping of...|153.99724039428568|
|Signage Requied f...|151.63868055333333|
|Storage of Used M...|142.11255641500003|
|Zoning: Recycle Yard| 135.9285161247979|
|Donation Containe...|131.75610506358709|
|License Requied U...|128.79828704142858|
|Traffic Signal Gr...|101.79846062200002|
|           Complaint| 72.87050230311695|
+--------------------+------------------+
only showing top 10 rows



### Does number of days late depend on department?

In [49]:
(df.filter('dept_name is not NULL')
   .filter('num_days_late is not NULL')
   .groupBy('dept_name').agg(f.mean('num_days_late').alias('avg_days_late'))
   .sort(f.col('avg_days_late').desc()).show(10))

+--------------------+-------------------+
|           dept_name|      avg_days_late|
+--------------------+-------------------+
|    Customer Service|  59.73709149630077|
|Development Services| 13.433724555869714|
|Solid Waste Manag...| -2.200057513672164|
|        Metro Health| -4.911766979607004|
|Parks and Recreation| -5.251521960055145|
|Trans & Cap Impro...|-20.612837354052708|
|Code Enforcement ...| -38.70133068329591|
|Animal Care Services|-226.51783940550334|
+--------------------+-------------------+



### How do number of days late depend on department and request type?

In [50]:
(df.filter('dept_name is not NULL')
   .filter('num_days_late is not NULL')
   .filter('service_request_type is not NULL')
   .groupBy('dept_name', 'service_request_type').agg(f.mean('num_days_late').alias('avg_days_late'))
   .sort(f.col('avg_days_late').desc()).show(10))

+--------------------+--------------------+------------------+
|           dept_name|service_request_type|     avg_days_late|
+--------------------+--------------------+------------------+
|Code Enforcement ...|  Zoning: Junk Yards|175.95636210420943|
|Code Enforcement ...|Labeling for Used...|162.43032902285717|
|Code Enforcement ...|Record Keeping of...|153.99724039428568|
|Code Enforcement ...|Signage Requied f...|151.63868055333333|
|Code Enforcement ...|Storage of Used M...|142.11255641500003|
|Code Enforcement ...|Donation Containe...|131.75610506358706|
|Code Enforcement ...|License Requied U...|128.79828704142858|
|Trans & Cap Impro...|Traffic Signal Gr...|101.79846062200002|
|    Customer Service|           Complaint|  72.8705023031169|
|Code Enforcement ...|             Vendors| 66.54809898507798|
+--------------------+--------------------+------------------+
only showing top 10 rows



In [ ]:
(df.filter('dept_name is not NULL')
   .filter('num_days_late is not NULL')
   .filter('service_request_type is not NULL')
   .groupBy('dept_name', 'service_request_type').agg(f.mean('num_days_late').alias('avg_days_late'))
   .sort(f.col('avg_days_late').desc()).show(10))

In [51]:
(df.filter('dept_name is not NULL')
   .filter('num_days_late is not NULL')
   .filter('service_request_type is not NULL')
   .groupBy('dept_name', 'service_request_type').agg(f.mean('num_days_late').alias('avg_days_late'))
   .sort(f.col('avg_days_late').asc()).show(10))

+--------------------+--------------------+-------------------+
|           dept_name|service_request_type|      avg_days_late|
+--------------------+--------------------+-------------------+
|Trans & Cap Impro...|  Engineering Design|      -1399.1272335|
|Trans & Cap Impro...|Signal Timing Mod...|-1247.0797799732143|
|Animal Care Services|        Stray Animal| -998.8064665118969|
|Parks and Recreation|Major Park Improv...| -280.2546235360405|
|Trans & Cap Impro...|Sidewalk Cost Sha...|-186.18202610536574|
|Code Enforcement ...|Multi Tenant Exte...| -135.7158812804762|
|Code Enforcement ...|   CPS Energy Towers|-129.84778717829747|
|Code Enforcement ...|CPS Energy Wood P...|-129.30905202721226|
|Code Enforcement ...|CPS Energy Metal ...| -129.1791978642777|
|Code Enforcement ...|Multi Tenant Inte...| -125.1431856354651|
+--------------------+--------------------+-------------------+
only showing top 10 rows

